# Mouse chimera proof-of-concept

In [ ]:
import numpy as np
import pandas as pd
import anndata

In [1]:
## r2py setup
import rpy2.rinterface_lib.callbacks
import logging
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)

import anndata2ri
anndata2ri.activate()